In [1]:
import pandas as pd
from src.utils import EncoderDecoder
import torch
import plotly.express as px
from sklearn.manifold import TSNE
from src.models.autoencoder import SimpleAutoEncoder

In [2]:
df = pd.read_csv("data/raw/all_climbs.csv", index_col=0)
encdec = EncoderDecoder()

In [3]:
model = SimpleAutoEncoder.load_from_checkpoint("logs/lightning_logs/9tdgfelc/checkpoints/epoch=46-step=7003.ckpt").cpu()

In [7]:
t = model.encode(encdec(df.iloc[0]['frames'], df.iloc[0]['angle']).unsqueeze(0))

In [10]:
subset = df.sort_values("ascensionist_count", ascending=False).head(5000)

In [15]:
encodings = []
for name, row in subset.iterrows():
    encodings.append(model.encode(encdec(row['frames'], row['angle']).unsqueeze(0)).squeeze(0).detach())
encodings = torch.stack(encodings, dim=0)

In [17]:
tsne = TSNE()
x_y = tsne.fit_transform(encodings)
subset['x'] = x_y[0,:]
subset['y'] = x_y[1,:]

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.